In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

# 기본 import

In [2]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate

# Memory

https://python.langchain.com/api_reference/langchain/memory.html

챗봇으로 하여금 대화(상태)를 '기억'하게끔 한다

Memory maintains Chain state, incorporating context from past runs.


In [3]:
# Langchain 의 memory 계층
#  BaseMemory --> BaseChatMemory --> <name>Memory  # Examples: ZepMemory, MotorheadMemory

In [4]:
# OpenAI 사에서 제공하는 '기본 API' 도 랭체인 없이 사용 가능.
# 메모리 지원하지 않는다.  이전 대화 기억 못함.  stateless 하다!

# ChatGPT 서비스 는 '메모리' 기능이 탑재되어 있다.
# 챗봇이 이전의 대화 내용과 질문을 기억하고 답할수 있다.


# 1.ConversationBufferMemory

In [5]:
# ConversationBufferMemory
# 대화 내용 '전체'를 저장하는 메모리

# 장점: 단순하다

# 단점:
# => 매번 요청할때마다 '이전 대화 기록 전체' 를 같이 보내야 함.
#  그래야 모델이 전에 일어났던 대화를 보고 이해 할수 있다.
#  대화내용이 길어질수록 메모리도 계속 커지니까 성능적으로도 & 비용적으로도 비효율적이다.


In [6]:
from langchain.memory.buffer import ConversationBufferMemory

In [7]:
memory = ConversationBufferMemory()

# 직접 save
memory.save_context(
    {'input': 'Hi!'},   # 유저 입력값
    {'output': 'How are you?'}, # AI 가 유저에게 답한 내용 기록
)

memory.load_memory_variables({})   # inputs= 매개변수가 빈dict


# {'history': 'Human: Hi!\nAI: How are you?'}
#  history 가 텍스트다


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16376\2708910790.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': 'Human: Hi!\nAI: How are you?'}

## return_messages=True
history 에  AIMessage 와 HumanMessage 로 저장된다.

In [8]:
memory = ConversationBufferMemory(return_messages=True)

memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True)

In [9]:
memory.save_context(
    {'input': 'Hi!'},   # 유저 입력값
    {'output': 'How are you?'}, # AI 가 유저에게 답한 내용 기록
)

memory.load_memory_variables({})

# {'history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
#   AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]}

#  history 가 Message 의 list 다!

{'history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]}

In [10]:
memory.save_context(
    {'input': 'Hi!'},   # 유저 입력값
    {'output': 'How are you?'}, # AI 가 유저에게 답한 내용 기록
)

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]}

# 2.ConversationBufferWindowMemory

In [11]:
from langchain.memory.buffer_window import ConversationBufferWindowMemory

In [12]:
# ConversationBufferWindowMemory 는 대화의 '특정 부분만' 을 저장하는 메모리.

# 장점:
#   메모리를 특정 크기로 유지할 수 있다!
#   따라서 모든 대화 내용을 저장하지 않아도 된다!

# 단점:
#   챗봇이 전체 대화가 아닌 '최근 대화' 에만 집중하게 된다.


In [13]:
memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,   # 버퍼 윈도우 사이즈.  몇개의 context(메세지)를 저장할지 지정
)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16376\823005658.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [14]:
# 도우미 함수 준비.
def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

In [15]:
# 동작 확인
add_message("1", "1")
add_message("2", "2")
add_message("3", "3")
add_message("4", "4")

In [16]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='3', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='4', additional_kwargs={}, response_metadata={}),
  AIMessage(content='4', additional_kwargs={}, response_metadata={})]}

In [17]:
add_message("5", "5")
memory.load_memory_variables({})

{'history': [HumanMessage(content='2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='3', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='4', additional_kwargs={}, response_metadata={}),
  AIMessage(content='4', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='5', additional_kwargs={}, response_metadata={}),
  AIMessage(content='5', additional_kwargs={}, response_metadata={})]}

# 3.ConversationSummaryMemory

In [18]:
# ConversationSummaryMemory 는 LLM 을 사용하여 대화의 요약본 (summary) 생성

In [19]:
llm = ChatOpenAI(temperature=0.1)

In [20]:
from langchain.memory.summary import ConversationSummaryMemory

In [21]:
# ConversationSummaryMemory
# 메세지를 그대로 저장하는 것이 아니라 Convertaion 의 '요약'을 해준다. LLM 필요

# 장점: 대화의 메세지가 많아질수록 요약을 해주어 토큰의 양도 줄어들어 훨씬 경제적인 방법
# 단점: LLM 호출 발생

memory = ConversationSummaryMemory(llm=llm)  # llm 을 사용하여 요약한다!



C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16376\2664529100.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)  # llm 을 사용하여 요약한다!


In [22]:
def get_history():
    return memory.load_memory_variables({})

In [23]:
# message 추가
add_message(
    "Hi I'm John, I live in South Korea",    # input
    "Wow that is so cool!"  # output : AI 답변
    )

In [24]:
# 또 message 추가
add_message(
    "South Korea is so pretty",
    "I wish I could go!!!")

In [25]:
get_history()

{'history': 'John introduces himself as living in South Korea. The AI responds with enthusiasm, finding it cool and expressing a desire to visit because South Korea is so pretty.'}

# 4.ConversationSummaryBufferMemory

In [26]:
from langchain.memory.summary_buffer import ConversationSummaryBufferMemory

In [27]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,   # 최대 가용한 메세지 토큰수 (메세지들이 요약되지 전)
    return_messages=True,
)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16376\2068819690.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [28]:
add_message(
    "Hi I'm John, I live in South Korea",    # input
    "Wow that is so cool!"  # output : AI 답변
    )
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={})]}

In [29]:
# 다시 메세지 추가하고 history 확인
add_message(
    "South Korea is so pretty",
    "I wish I could go!!!")
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={})]}

In [30]:
add_message(
    "How far is Korea from Argentina?",
    "I don't know! Super far!"
)
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Korea from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={})]}

In [31]:
# 아래 셀을 여러차레 해보자  약 (3,4번?)
# =>실제 '요약'이 발생할때면 Model IO 가 발생하기 때문에 시간이 좀 걸리는게 느껴질거다!
add_message(
    "How far is Brazil from Argentina?",
    "I don't know! Super far!"
)
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Korea from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Brazil from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={})]}

In [32]:
# max_token_limit 에 도달하면, 오래된 메세지들이 요약되고 있을 것을 확인할수 있다. (SystemMessage 확인)

# ★그러나 '요약' 이라는 과정은 API 를 사용한다는 사실을 명심하세요.
# ★'요약' 동작은 비용 지출이 발생되는 부분입니다.


# 5.ConversationKGMemory
KG : Knowledge Graph

In [33]:
# 대화중에 '엔티티'의 knowledge graph 를 형성한다 => 가장 중요한 것들만 추출한 요약본.
# knowledge graph 는 history 를 가지고 오지 않는다.  대신 '엔티티' 를 가지고 옴


In [34]:
from langchain_community.memory.kg import ConversationKGMemory

In [35]:
memory = ConversationKGMemory(
    llm=llm,  # 이 또한 LLM 을 사용하는 Memory 다  -> Knowledge Graph 를 만든다. (가장 중요한것만 뽑아낸 요약본)
    return_messages=True,
)

In [36]:
add_message(
    "Hi I'm John, I live in South Korea",    # input
    "Wow that is so cool!"  # output : AI 답변
    )

In [37]:
# 대화의 특정 entity 에 대해 질무ㄴ해보자
#  Model IO 발생
memory.load_memory_variables({"input": "Who is John"})   # 빈dict 가 아니다!

{'history': [SystemMessage(content='On John: John lives in South Korea.', additional_kwargs={}, response_metadata={})]}

In [38]:
add_message("John likes kimchi", "Wow that is so cool!")

In [39]:
memory.load_memory_variables({"input": "Who is John"})

{'history': [SystemMessage(content='On John: John lives in South Korea. John likes kimchi.', additional_kwargs={}, response_metadata={})]}

# 6. 참고: Database 와 integration 된 메모리들

- **참고: Database 와 integration 된 메모리들**

| Memory Class                  | 통합 대상 (Integration)             | 설명                                                           |
| ----------------------------- | ------------------------------- | ------------------------------------------------------------ |
| `RedisChatMessageHistory`     | **Redis**                       | Redis에 메시지 저장. 빠르고 확장 가능한 저장소.                               |
| `SQLChatMessageHistory`       | **SQLite, PostgreSQL 등 SQL DB** | SQL 데이터베이스에 메시지 저장. SQLAlchemy 기반.                           |
| `DynamoDBChatMessageHistory`  | **AWS DynamoDB**                | AWS의 NoSQL DB인 DynamoDB에 대화 저장. 서버리스 환경에서 유용.                |
| `MongoDBChatMessageHistory`   | **MongoDB**                     | 문서 기반 DB인 MongoDB와 통합하여 대화 저장.                               |
| `PostgresChatMessageHistory`  | **PostgreSQL**                  | PostgreSQL 전용 구현 (SQLAlchemy 없이).                            |
| `FileChatMessageHistory`      | **Local 파일**                    | JSON 파일로 로컬에 저장. 간단한 로깅에 적합.                                 |
| `FirestoreChatMessageHistory` | **Google Firestore**            | Firebase 기반 클라우드 NoSQL DB와 통합.                               |
| `ChromaMemory`                | **Chroma (벡터 DB)**              | 벡터 DB에 embedding 형태로 memory 저장. RAG나 유사 검색 기반 memory에 활용 가능. |
| `WeaviateMemory`              | **Weaviate**                    | Weaviate 벡터 DB와 통합된 memory 저장.                               |
| `QdrantMemory`                | **Qdrant**                      | 벡터 기반 memory 저장소로 Qdrant 사용.                                 |


# 7.Memory on LLMChain

In [40]:
# 메모리를 'chain' 에 꽂는 방법.
# 두가지 형태의 'chain' 에 각각 꽂는 방법.
#    1. off-the-shelf chain  : LangChain 에서 '일반적인 목적'을 수행하는 기본 제공되는 chain
#    2. LCEL 사용한 chain : 커스텀 chains


In [41]:
from langchain.chains.llm import LLMChain

In [42]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=80,
)

chain = LLMChain(   # prompt | llm
    llm=llm,
    memory=memory,  # 메모리 제공
    prompt=PromptTemplate.from_template("{question}")
)



C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16376\2707029833.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(   # prompt | llm


In [43]:
chain.invoke(input={"question": "My name is John"})

{'question': 'My name is John',
 'history': '',
 'text': 'Nice to meet you, John! How can I assist you today?'}

In [44]:
chain.invoke(input={"question": "I live in Seoul"})

{'question': 'I live in Seoul',
 'history': 'Human: My name is John\nAI: Nice to meet you, John! How can I assist you today?',
 'text': ", the capital city of South Korea. It is a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic palaces. I love exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views of the Han River. Seoul is a dynamic and exciting place to call home."}

In [45]:
# 과연 이름을 기억하고 있을까?
chain.invoke(input={'question':'What is my name?'})

{'question': 'What is my name?',
 'history': "System: The human introduces himself as John. The AI greets John and asks how it can assist him. John mentions that he lives in Seoul.\nAI: , the capital city of South Korea. It is a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic palaces. I love exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views of the Han River. Seoul is a dynamic and exciting place to call home.",
 'text': "I'm sorry, I do not have access to that information."}

In [46]:
# 이름을 모른다고???

# chain 을 디버깅해보자.

## verbose=
chain 을 실행했을때 chain 의 프롬프트와 로그들을 확인할수 있다. (디버깅용)

In [47]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True,  # chain 을 실행했을때 chain 의 프롬프트 로그들을 확인할수 있다. (디버깅용)
)

chain.invoke(input={'question':"My name is John"})
chain.invoke(input={'question':"I live in Seoul"})
chain.invoke(input={'question':"What is my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
My name is John

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
I live in Seoul

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
What is my name?

> Finished chain.


{'question': 'What is my name?',
 'history': "System: John introduces himself and mentions that he lives in Seoul, the capital city of South Korea. The AI responds by describing Seoul as a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic palaces. The AI loves exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views of the Han River, making Seoul a dynamic and exciting place to call home. The human reveals his name is John, and the AI warmly greets him and continues to praise Seoul for its energy and charm.",
 'text': "I'm sorry, I do not have access to personal information such as your name."}

In [48]:
# ↑ chain 의 prompt 로그 들을 확인할 수 있다.
# 보다시피 대화의 내역(history) 가 prompt에 계속 추가되진 않는다.

# 우리가 원하는 어떤 방식으로 prompt에게 대화 기록(history)을 추가해줘야 한다!

# ↓ 하지만! 메모리는 계속 업데이트 된다.  함 보자!

In [49]:
memory.load_memory_variables({})

{'history': "System: John introduces himself and mentions that he lives in Seoul, the capital city of South Korea. The AI responds by describing Seoul as a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic palaces. The AI loves exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views of the Han River, making Seoul a dynamic and exciting place to call home. The human reveals his name is John, and the AI warmly greets him and continues to praise Seoul for its energy and charm.\nHuman: What is my name?\nAI: I'm sorry, I do not have access to personal information such as your name."}

In [50]:
# 위 메모리 내용이 prompt 에 포함되어야 한다!

# memory_key= 에 "chat_history" 라고 말해주어야 한다.

## memory_key=

In [51]:
memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history", 
)

# ↓ template 안에는 memory 가 history 를 저장하도록 한 곳을 적어주기만 하면 된다
# history 까지 담을 괜찮은 템플릿을 준비해보자
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""
# AI 가 우리의 대화기록을 기억하면서 여기의 question 을 완성할 수 있기를 기대해보자.

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),  # <- template 지정
    verbose=True,  # 어떤 prompt 가 만들어질까?  두근두근
)

In [52]:
chain.invoke({'question': "My name is John"})



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is John
    You:


> Finished chain.


{'question': 'My name is John',
 'chat_history': '',
 'text': 'Hello John! How can I assist you today?'}

In [53]:
chain.invoke({'question': "I live in Seoul"})



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is John
AI: Hello John! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


{'question': 'I live in Seoul',
 'chat_history': 'Human: My name is John\nAI: Hello John! How can I assist you today?',
 'text': "That's great to hear! How can I assist you with anything related to Seoul or anything else today?"}

In [54]:
chain.invoke({'question':'What is my name?'})



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is John
AI: Hello John! How can I assist you today?
Human: I live in Seoul
AI: That's great to hear! How can I assist you with anything related to Seoul or anything else today?
    Human:What is my name?
    You:


> Finished chain.


{'question': 'What is my name?',
 'chat_history': "Human: My name is John\nAI: Hello John! How can I assist you today?\nHuman: I live in Seoul\nAI: That's great to hear! How can I assist you with anything related to Seoul or anything else today?",
 'text': 'Your name is John.'}

In [55]:
# - 프롬프트 템플릿 안에서 메모리 내용이 들어갈 공간을 준비한다.  (예: chat_history)
# - 메모리를 활용할 템플릿은 원하는대로 작성하면 된다.
# - Memory 클래스에선 history 를 어디에 꽂을지 지정해준다 (memory_key=)


# 8.Chat based Memory

In [56]:
# memory 클래스는 memory 를 두가지 방식으로 출력
#  1. 문자열 형태
#  2. Message 형태

In [57]:
memory.load_memory_variables({})
# ↓ 결과는 텍스트 형태 (문자열).  프롬프트에 표시되는 방식도 텍스트다.

{'chat_history': "Human: My name is John\nAI: Hello John! How can I assist you today?\nHuman: I live in Seoul\nAI: That's great to hear! How can I assist you with anything related to Seoul or anything else today?\nHuman: What is my name?\nAI: Your name is John."}

## return_messages=True

In [58]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key='chat_history',
    return_messages=True,  # history 를 '문자열' 이 아닌 'Message' 로 바꾸어 리턴함.
)

In [59]:
from langchain_core.prompts.chat import ChatPromptTemplate

In [ ]:
# 과연 prompt 에는 어떻게 대화 history 들을 넘겨줄수 있을까?
#  단순한 하나의 텍스트가 아니라... Human Message - AI Message - Human Messagbe - AI Message - .... (여러 메세지들)
#  심지여 요약본 발생시 System message 도 있을텐데?

# prompt 에 이를 위한 공간을 어케 만드나?
#  => MessagePlaceHolder

## MessagePlaceHolder

In [60]:
from langchain_core.prompts.chat import MessagesPlaceholder

In [61]:
# MessagesPlaceholder 를 활용한 프롬프트 를 작성
prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a helpful AI talking to a human"),

    MessagesPlaceholder(variable_name='chat_history'),
    
    ('human', '{question}'),
])

In [62]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt, 
    verbose=True,
)

In [63]:
chain.invoke({'question':"My name is John"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is John

> Finished chain.


{'question': 'My name is John',
 'chat_history': [HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'text': 'Hello John! How can I assist you today?'}

In [64]:
chain.invoke({'question':"I live in Seoul"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is John
AI: Hello John! How can I assist you today?
Human: I live in Seoul

> Finished chain.


{'question': 'I live in Seoul',
 'chat_history': [HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I live in Seoul', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?', additional_kwargs={}, response_metadata={})],
 'text': 'Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?'}

In [65]:
chain.invoke({'question':"What is my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is John
AI: Hello John! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?
Human: What is my name?

> Finished chain.


{'question': 'What is my name?',
 'chat_history': [HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I live in Seoul', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is John.', additional_kwargs={}, response_metadata={})],
 'text': 'Your name is John.'}

# LCEL Based Memory

In [66]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key='chat_history',
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


In [67]:
chain = prompt | llm

In [70]:
chain.invoke({
    'chat_history': memory.load_memory_variables({})['chat_history'],
    'question': 'My name is John',
})

AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CaJNvmoYnC7woBYCpmpE9Nzf0IXwN', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--5548febc-550f-4730-a51c-9c338e31b0ba-0', usage_metadata={'input_tokens': 24, 'output_tokens': 10, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [71]:
# 위 방법도 가능하지만,  Runnable 을 사용하여 호출해보자.

## RunnablePassThrough

In [72]:
from langchain_core.runnables.passthrough import RunnablePassthrough

In [73]:
def load_memory():
    return memory.load_memory_variables({})['chat_history']

In [74]:
chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

# 위 chain 을 호출하면 load_memory() 가 가장 먼처 호출.
#  호출 결과가 chat_history 키값에 담겨 prompt 로 전달.

# 이는 마치
# chain.invoke({
#     "chat_history": load_memory(),  <- 요렇게 한것과 동일하다.
#     "question": "My name is John",
# })

chain.invoke({
    "question": "My name is John",
})


TypeError: load_memory() takes 0 positional arguments but 1 was given

In [75]:
# 도대체 어떤값이 load_memory() 에 전달되는지 확인해보자!

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def load_memory(input):
  print("🎃load_memory()", input)  # 확인해보자!!
  return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

chain.invoke({
    "question": "My name is John",
})

🎃load_memory() {'question': 'My name is John'}


AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CaJUlrOVpGgvpycaEOJRzwOUdKkjJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--158b5305-f850-4b1e-a161-c15bdbd97dc1-0', usage_metadata={'input_tokens': 24, 'output_tokens': 10, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
"""
load_memory() {'question': 'My name is John'}

체인에 있는 모든 컴포넌트는 input 을 받을거고, 또 output 을 줄거다.
이게 랭체인의 핵심이다.  모든 것이 input을 얻을거고, 그 후엔 output 을 줄거다.


chain.invoke(
  {     <--  이 dict 는 chain 의 첫번째 아이템의 input 이 되는거다.
             바로 그게 load_memory() 의 input 이 된것이다!   규칙이다!
    "question": "My name is John",
  }
)

이후 load_memory() 를 실행히킨 결과로 얻은것이 chat_history 속성으로 들어가
체인의 다음 요소 로 전달되는 것이다.

RunnablePassThrough 는 최초 전달받은 'question' 을 체인의 다음요소 까지 전달 (key 값 변경 없이!)

"""
None



In [ ]:
"""
여기서 또 한가지 문제는, chain 호출후 그 결과를 다시 메모리에 저장해야 한다는 것이다.
(지금은 메모리 관리를 수동으로 하고 있다.)

↓ 여기서 가장 좋은 방법은 체인을 호출하는 함수를 만들어 보는 것이다.
"""
None

In [76]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def load_memory(input):
  print("🎃load_memory()", input)
  return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

# 체인 호출 함수를 직접 만들어 보자
def invoke_chain(question):
    result = chain.invoke({"question": question})
    # 사용자 질문(question) 과 체인 호출 결과 (result) 를 메모리에 저장
    memory.save_context(
        {'input': question},
        {'output': result.content},
    )
    print(result)

In [77]:
invoke_chain("My name is John")

🎃load_memory() {'question': 'My name is John'}
content='Hello John! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CaJcBJckXU0OuzVZEOCalnF5VmGrQ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--f760089e-6a31-4e6f-8b88-44987432b8c9-0' usage_metadata={'input_tokens': 24, 'output_tokens': 10, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [78]:
invoke_chain("What is my name?")

🎃load_memory() {'question': 'What is my name?'}
content='Your name is John.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 47, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CaJcZs7DRWkJ2QVLeUx4zvc7OfLNU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--7b2fadf9-47fc-458b-bcdc-69290eae2c7d-0' usage_metadata={'input_tokens': 47, 'output_tokens': 5, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [79]:
get_history()

{'chat_history': [HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello John! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is John.', additional_kwargs={}, response_metadata={})]}

## memory_key= 삭제

In [ ]:
# memory = ConversationSummaryBufferMemory(
#     llm=llm,
#     max_token_limit=120,
#     memory_key="chat_history",  <-- 이제 이것도 필요하지 않을거다
#                   memory_key= 의 기본값은 "history" 이기 때문에
#                   모든 chat_history 를 이걸로 대체할수 있다.
#     return_messages=True,
# )

In [80]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    # memory_key= 삭제.  기본값 'history'
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),  # variable_name= 값을 기본값 'history'
        ("human", "{question}"),
    ]
)

def load_memory(input):
  print("🎃load_memory()", input)
  return memory.load_memory_variables({})["history"]  # 변경!

                                    # ↓ 변경!
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {'input': question},
        {'output': result.content},
    )
    print(result)

In [81]:
invoke_chain("My name is John")

🎃load_memory() {'question': 'My name is John'}
content='Hello John! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CaJi98UaPBZMi9r5Yb5EfMTKxbloY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--1cc1b101-ae98-43ab-a6d5-3566f1ac3fb9-0' usage_metadata={'input_tokens': 24, 'output_tokens': 10, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [82]:
invoke_chain("What is my name?")

🎃load_memory() {'question': 'What is my name?'}
content='Your name is John.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 47, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CaJiMY4g1H0yLsXATqqf8k27mJHz9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--37acd837-46ae-4e56-a223-2030aa22766b-0' usage_metadata={'input_tokens': 47, 'output_tokens': 5, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
